In [ ]:
%matplotlib inline
import os
import time
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime

from caen_felib import lib, device, error

# Get CAEN FELib path
print(f'CAEN FELib found at: {lib.path} (version={lib.version})')

In [ ]:
from lgdo import lh5, Table, Array, WaveformTable, ArrayOfEqualSizedArrays

## Connect

In [ ]:
#dig = device.connect("dig2://192.168.0.254")
dig = device.connect("dig2://caendgtz-usb-52696")
print(f'Connected to digitizer (handle={hex(dig.handle)}, name={dig.name})')

In [ ]:
tree = dig.get_device_tree()

In [ ]:
#for par, par_dict in tree["par"].items():
for par in tree["ch"]["0"]["par"]:
    if isinstance(par_dict, dict) and "accessmode" in par_dict:
        #if par_dict['accessmode']['value'] == "READ_ONLY":#"READ_WRITE":
        alv = []
        if 'allowedvalues' in par_dict:
            allowedvalues = par_dict['allowedvalues']
            for a in allowedvalues:
                if isinstance(allowedvalues[a], dict):
                    alv.append(allowedvalues[a]["value"])
        def_val = ""
        if 'defaultvalue' in par_dict:
            def_val = par_dict['defaultvalue']['value']
        print(f"  {par}: {def_val} # ({par_dict['datatype']['value']}) [{alv}] {par_dict['description']['value']}")

In [ ]:
# Reset
dig.cmd.Reset()

fw_type = dig.par.fwtype.value
fw_ver = dig.par.fpga_fwver.value
print(fw_type, fw_ver)

adc_samplrate_msps = float(dig.par.adc_samplrate.value)  # in Msps
adc_n_bits = int(dig.par.adc_nbit.value)
sampling_period_ns = int(1e3 / adc_samplrate_msps)

print(f"Sampling rate = {adc_samplrate_msps} MHz, n. bit = {adc_n_bits}, Sampling period = {sampling_period_ns} ns")

# Configuration parameters
nch = int(dig.par.NumCh.value)
active_ch = 1
#reclen = 4084 #sample
reclen = 125000 #sample
pretrg = 2042 #sample
dc_offset = 10

dig.par.iolevel.value = "TTL"
dig.par.recordlengths.value = f"{reclen}"
dig.par.pretriggers.value = f"{pretrg}"

#dig.par.acqtriggersource.value = "TrgIn"
dig.par.acqtriggersource.value = "SwTrg"

# Set some channel parameters
for i, ch in enumerate(dig.ch):
    ch.par.chenable.value = "TRUE" if i < active_ch else "FALSE"  # Enable only channel 0
    ch.par.dcoffset.value = f"{dc_offset}"
    if i < active_ch:
        print(ch.par["dcoffset"].value)

In [ ]:
ch.set_value("

## Configure endpoint

In [ ]:
data_format = [
    {
        'name': 'EVENT_SIZE',
        'type': 'SIZE_T',
    },
    {
        'name': 'TIMESTAMP',
        'type': 'U64',
    },
    {
        'name': 'WAVEFORM',
        'type': 'U16',
        'dim': 2,
        'shape': [active_ch, reclen],
    },
    {
        'name': 'WAVEFORM_SIZE',
        'type': 'U64',
        'dim': 1,
        'shape': [active_ch],
    },
]

In [ ]:
decoded_endpoint_path = "scope"
endpoint = dig.endpoint[decoded_endpoint_path]
data = endpoint.set_read_data_format(data_format)
dig.endpoint.par.ACTIVEENDPOINT.value = decoded_endpoint_path

event_size = data[0].value
timestamp = data[1].value
waveform = data[2].value
waveform_size = data[3].value

In [ ]:
nev = 10
timestamps = np.zeros((active_ch,nev),dtype=np.uint64)
wfs = np.zeros((active_ch,nev,reclen),dtype=np.uint16)
temp_names = ["tempsensfirstadc","tempsenshottestadc","tempsenslastadc","tempsensairin","tempsensairout","tempsenscore","tempsensdcdc"]
temperatures = np.zeros((nev,len(temp_names)),dtype=float)

In [ ]:
dig.cmd.armacquisition()
dig.cmd.swstartacquisition()
for i in range(nev):
    time.sleep(1/1000)
    dig.cmd.sendswtrigger() # to send software trigger
    try:
        endpoint.read_data(-1, data)
    except error.Error as ex:
        if ex.code == error.ErrorCode.TIMEOUT:
            continue
        if ex.code == error.ErrorCode.STOP:
            break
        else:
            raise ex
    for j in range(active_ch):
        wfs[j,i] = waveform[j]
        timestamps[j,i] = timestamp
        for k, temp in enumerate(temp_names):
            temp_value = float(dig.get_value(f"/par/{temp}"))
            temperatures[i][k] = temp_value
dig.cmd.disarmacquisition()

# read files

In [ ]:
abs_lh5_file = "../tmp/test_20250601T144850Z.lh5"
total_channels = 64
channels = lh5.ls(abs_lh5_file)
plot_channels = []
for i in range(total_channels):
    chn = f"ch{i:03d}"
    if chn in channels:
        plot_channels.append(chn)
ncols = 4
nrows = len(plot_channels) // ncols + (len(plot_channels) % ncols > 0)

In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(24, 6 * nrows))
axes_flat = axes.flatten()

for idx, chn in enumerate(plot_channels):
    ax = axes_flat[idx]
    n_total_rows = lh5.read_n_rows(f"{chn}/raw", abs_lh5_file)
    if n_total_rows == 0:
        ax.set_title(f"{chn} (No WFs)")
        continue
    row_offset = max(0, n_total_rows - 1)
    raw_data = lh5.read(f"{chn}/raw", abs_lh5_file, start_row=row_offset, n_rows=1)
    if raw_data is None or not hasattr(raw_data, 'waveform') or raw_data.waveform is None:
        ax.set_title(f"{chn} (No WFs)")
        continue
    wfs = raw_data.waveform.values.nda
    dt = raw_data.waveform.dt.nda[0] / 1000. # us
    timestamp = raw_data.timestamp.nda
    dts = np.linspace(0, (wfs.shape[1]-1) * dt, wfs.shape[1])
    if wfs.shape[0] == 1 and wfs.shape[1] > 0:
        date = datetime.fromtimestamp(timestamp[0]/1e9).strftime("%Y-%m-%d %H:%M:%S")
        ax.plot(dts, wfs[0])
        ax.set_xlim(0, dts[-1])
        ax.set_xlabel(r"Time ($\mu$s)")
        ax.set_ylabel("ADC")
        ax.set_title(f"{chn} (Last Event at {date})")
    else:
        ax.set_title(f"{chn} (No Last Event)")

In [ ]:
from scipy.signal import periodogram

In [ ]:
nev, wsize = wfs0.shape
sampling_rate =  1 / 8 * 1e9 # rate in Hz
for j in range(nev):
    (freq, psd_tmp) = periodogram(wfs0[j], sampling_rate, scaling='density')
    if j == 0: psd = psd_tmp
    else: psd += psd_tmp
psd = np.array(psd / nev)
psd_db = 10 * np.log10(psd)
freq = np.array(freq)
rms_total = np.sqrt(np.trapz(psd, freq))

In [ ]:
plt.plot(freq[1:]/1e6, psd[1:], label=f"RMS = {rms_total:.2f} LSB")
plt.ylabel(r"Power Spectral Density ([ADC$^2$/Hz])")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Frequency (MHz)")

# Reading raw files

In [ ]:
raw_file = "/mnt/c/Users/valda/data/test_pulser_buffers/20241128_hc_buffer/pulser_pb5_400mV_100Hz_initest_000.lh5"
dig_data = lh5.read("dig/raw",raw_file)
temp0 = dig_data.temp0.nda
raw_data0 = lh5.read("ch000/raw",raw_file)
raw_data1 = lh5.read("ch001/raw",raw_file)
#raw_data2 = lh5.read("ch002/raw",raw_file)
#raw_data3 = lh5.read("ch003/raw",raw_file)
#raw_data4 = lh5.read("ch004/raw",raw_file)
timestamp = raw_data0.timestamp.nda
wfs0 = raw_data0.waveform.values.nda
wfs1 = raw_data1.waveform.values.nda
#wfs2 = raw_data2.waveform.values.nda
#wfs3 = raw_data3.waveform.values.nda
#wfs4 = raw_data4.waveform.values.nda

In [ ]:
dts = np.linspace(0,0.008*len(wf0),len(wf0))

In [ ]:
for i in range(1):
    fig, ax = plt.subplots(figsize=(8,4.5))
    ts = timestamp[i] - timestamp[0]
    wf0 = wfs0[i]# - np.mean(wfs[i,:1000])
    wf1 = wfs1[i]# - np.mean(wfs1[i,:1000])
    #wf2 = wfs2[i]# - np.mean(wfs1[i,:1000])
    #wf3 = wfs3[i]# - np.mean(wfs1[i,:1000])
    #wf4 = wfs4[i]# - np.mean(wfs1[i,:1000])
    ax.plot(dts,wf0,label="output 0")
    ax.plot(dts,wf1+1000,label="output 1")
    #ax.plot(dts,wf2+2000,label="output 2")
    #ax.plot(dts,wf3+3000,label="output 3")
    #ax.plot(dts,wf4+4000,label="output 4")
    ax.set_xlim(16.8,17.8)
    ax.legend(title=f"time={ts}, T = {temp0[i]} °C")
    ax.set_xlabel("Time ($\mu$s)")

In [ ]:
fig, ax = plt.subplots(figsize=(12,6.75))
#ax.plot(voltages,wf_max,"b.")
pars = np.polyfit(voltages,wf_max,1)
ax.plot(wf_max,wf_max-np.polyval(pars,voltages),"b.")
ax.axhline(0,color="k",ls=":")
ax.set_ylabel("Residuals (LSB)")
ax.set_xlabel("ADC channels (LSB)")